In [1]:
import pandas as pd
import pickle
from pandas import DataFrame
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.feature_selection import SequentialFeatureSelector, RFE, SelectFromModel
import seaborn as sns

In [2]:
df = pd.read_csv("./dataset.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93312 entries, 0 to 93311
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       93312 non-null  float64
 1   Occupation                93312 non-null  int64  
 2   Annual_Income             93312 non-null  float64
 3   Monthly_Inhand_Salary     93312 non-null  float64
 4   Num_Bank_Accounts         93312 non-null  float64
 5   Num_Credit_Card           93312 non-null  float64
 6   Interest_Rate             93312 non-null  float64
 7   Num_of_Loan               93312 non-null  int64  
 8   Delay_from_due_date       93312 non-null  float64
 9   Num_of_Delayed_Payment    93312 non-null  float64
 10  Changed_Credit_Limit      93312 non-null  float64
 11  Num_Credit_Inquiries      93312 non-null  float64
 12  Credit_Mix                93312 non-null  int64  
 13  Outstanding_Debt          93312 non-null  float64
 14  Credit

In [3]:
def cross_val_metrics(clf, X, y, cv=5):
    accuracy = []
    f1 = []
    precision = []
    recall = []
    kf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=36)
    for train_idx, test_idx in kf.split(X, y):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        accuracy.append(metrics.accuracy_score(y_test, y_pred))
        precision.append(metrics.precision_score(y_test, y_pred, average=None))
        recall.append(metrics.recall_score(y_test, y_pred, average=None))
        f1.append(metrics.f1_score(y_test, y_pred, average=None))
    print("accuracy", np.mean(accuracy))
    print("precision", np.mean(precision, axis=0))
    print("recall",np.mean(recall, axis=0))
    print("f1",np.mean(f1, axis=0))

In [4]:
labels = ['Poor', 'Standard', 'Good']

In [5]:
X = df.drop(['Credit_Score'], axis=1)
y = df['Credit_Score']
X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=36)

print(X_train.shape, y_train.shape)

(74649, 30) (74649,)


In [6]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(pd.DataFrame(metrics.confusion_matrix(y_test, y_pred, labels=np.unique(y_test)), 
             index=['True ' + x for x in labels], 
             columns=['Predict ' + x for x in labels]))
print(metrics.classification_report(y_test, y_pred, target_names=labels))

               Predict Poor  Predict Standard  Predict Good
True Poor              3812              1365            78
True Standard          1450              7600          1030
True Good                82               972          2274
              precision    recall  f1-score   support

        Poor       0.71      0.73      0.72      5255
    Standard       0.76      0.75      0.76     10080
        Good       0.67      0.68      0.68      3328

    accuracy                           0.73     18663
   macro avg       0.72      0.72      0.72     18663
weighted avg       0.73      0.73      0.73     18663



In [7]:
cross_val_metrics(clf, X, y)

accuracy 0.7358003955486311
precision [0.72336067 0.76239781 0.67417215]
recall [0.72282737 0.76510966 0.66750863]
f1 [0.72308378 0.76373943 0.67074221]


Độ chính xác khi sử dụng DecisionTree là khoảng 71%, cải thiện 4% so với việc không xử lí dữ liệu rỗng hoặc sai.

Nhãn Good và Poor có f1-score cải thiện rõ rệt, có thể do việc xử lí dữ liệu làm các điểm phân cách giữa Standard và Good/Poor trở nên rõ ràng và chính xác hơn.

In [8]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_train)
print(pd.DataFrame(metrics.confusion_matrix(y_train, y_pred, labels=np.unique(y_train)), 
             index=['True ' + x for x in labels], 
             columns=['Predict ' + x for x in labels]))
print(metrics.classification_report(y_train, y_pred, target_names=labels))

               Predict Poor  Predict Standard  Predict Good
True Poor             21021                 0             0
True Standard             0             40318             0
True Good                 0                 0         13310
              precision    recall  f1-score   support

        Poor       1.00      1.00      1.00     21021
    Standard       1.00      1.00      1.00     40318
        Good       1.00      1.00      1.00     13310

    accuracy                           1.00     74649
   macro avg       1.00      1.00      1.00     74649
weighted avg       1.00      1.00      1.00     74649



Tuy nhiên, do số lượng feature đầu vào là rất nhiều (30 features), việc sử dụng một Decision Tree đơn giản có thể dễ dàng xảy ra overfit do dễ dàng tạo ra một cây quá đúng với tập train mà không khái quát được dữ liệu.

In [9]:
clf = RandomForestClassifier()
clf = clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(pd.DataFrame(metrics.confusion_matrix(y_test, y_pred, labels=np.unique(y_test)), 
             index=['True ' + x for x in labels], 
             columns=['Predict ' + x for x in labels]))
print(metrics.classification_report(y_test, y_pred, target_names=labels))

               Predict Poor  Predict Standard  Predict Good
True Poor              4384               863             8
True Standard          1210              8160           710
True Good                17               785          2526
              precision    recall  f1-score   support

        Poor       0.78      0.83      0.81      5255
    Standard       0.83      0.81      0.82     10080
        Good       0.78      0.76      0.77      3328

    accuracy                           0.81     18663
   macro avg       0.80      0.80      0.80     18663
weighted avg       0.81      0.81      0.81     18663



In [10]:
cross_val_metrics(clf, X, y)

accuracy 0.8178155809533594
precision [0.79534455 0.84012448 0.78862981]
recall [0.84209933 0.8217589  0.76752084]
f1 [0.81805154 0.83083459 0.77790318]


### Ưu điểm:
Kết quả được cải thiện rõ rệt sau khi phân tích xử lí dữ liệu và lựa chọn thuật toán phù hợp hơn với dữ liệu phức tap
### Nhược điểm
Số lượng feature vẫn rất nhiều khiến mô hình lớn và phức tạp